In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding, TimeDistributed, Attention
from sklearn.model_selection import train_test_split
import re, os


In [18]:
# -*- coding: utf-8 -*-
# Afficher les appareils disponibles
print("GPU disponible :", tf.config.list_physical_devices('GPU'))


GPU disponible : []


In [19]:
##Si l'archive n'est pas présente, on la télécharge et on la dézippe
if not os.path.exists("cornell_movie_dialogs_corpus.zip"):
  print("Downloading data (234 MB)")
  !wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
  !unzip cornell_movie_dialogs_corpus.zip


In [20]:
lines_path = "cornell movie-dialogs corpus/movie_lines.txt"
convs_path = "cornell movie-dialogs corpus/movie_conversations.txt"
def load_and_preprocess_data():
    # Charger le dataset Cornell Movie Dialogs
    # (Vous devrez adapter ceci selon la structure exacte de vos fichiers)
    lines = open(lines_path, encoding='utf-8', errors='ignore').read().split('\n')
    conv_lines = open(convs_path, encoding='utf-8', errors='ignore').read().split('\n')

    # Créer un dictionnaire pour mapper les ID de ligne au texte
    id2line = {}
    for line in lines:
        parts = line.split(' +++$+++ ')
        if len(parts) == 5:
            id2line[parts[0]] = parts[4]

    # Créer des paires question-réponse
    pairs = []
    for conv in conv_lines[:-1]:
        parts = conv.split(' +++$+++ ')
        if len(parts) == 4:
            line_ids = eval(parts[3])  # Convertir la string en liste
            for i in range(len(line_ids)-1):
                pairs.append((id2line[line_ids[i]], id2line[line_ids[i+1]]))

    return pairs

# Modifiez clean_text() pour moins agressif :
def clean_text(text):
    text = text.lower()
    # Garder la ponctuation basique
    text = re.sub(r"([.!?])", r" \1", text)
    # Conserver les contractions (améliore le naturel)
    text = re.sub(r"i'm", "i'm", text)
    text = re.sub(r"he's", "he's", text)
    # Ne supprimer que les caractères vraiment inutiles
    text = re.sub(r"[^a-zA-Z.!?']", " ", text)
    return text.strip()

# Charger et nettoyer les données
pairs = load_and_preprocess_data()
cleaned_pairs = [(clean_text(q), clean_text(a)) for q, a in pairs]

In [ ]:
# Paramètres
# Paramètres du modèle
MAX_LENGTH = 15
VOCAB_SIZE = 10000
EMBEDDING_DIM = 128  
LSTM_UNITS = 128  
BATCH_SIZE = 128  
EPOCHS = 10  

# Création du tokenizer
tokenizer = Tokenizer(
    num_words=VOCAB_SIZE,
    oov_token='<OOV>',
    filters=''  # Désactive les filtres par défaut
)
all_text = [q for q, a in cleaned_pairs] + [a for q, a in cleaned_pairs]
tokenizer.fit_on_texts(all_text)

# Ajout des tokens spéciaux
word_index = tokenizer.word_index
word_index['<start>'] = len(word_index) + 1
word_index['<end>'] = len(word_index) + 1
tokenizer.word_index = word_index
VOCAB_SIZE = len(word_index) + 1

In [22]:
def tokenize_and_pad(pairs):
    input_seqs = []
    target_seqs = []

    for q, a in pairs:
        # Tokeniser la question
        q_seq = tokenizer.texts_to_sequences([q])[0]
        q_seq = q_seq[:MAX_LENGTH-1]  # Tronquer si nécessaire
        input_seqs.append(q_seq)

        # Tokeniser la réponse avec tokens start/end
        a_seq = tokenizer.texts_to_sequences([a])[0]
        a_seq = a_seq[:MAX_LENGTH-1]
        a_seq = [word_index['<start>']] + a_seq + [word_index['<end>']]
        target_seqs.append(a_seq)

    # Appliquer le padding
    input_seqs = pad_sequences(input_seqs, maxlen=MAX_LENGTH, padding='post')
    target_seqs = pad_sequences(target_seqs, maxlen=MAX_LENGTH+1, padding='post')

    return input_seqs, target_seqs

input_seqs, target_seqs = tokenize_and_pad(cleaned_pairs)


In [23]:
# Encodeur
encoder_inputs = Input(shape=(MAX_LENGTH,))
encoder_embedding = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(encoder_inputs)
encoder_lstm = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)

state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Décodeur
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(decoder_inputs)
decoder_lstm = LSTM(LSTM_UNITS*2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Couche d'attention
attention = Attention()([decoder_outputs, encoder_outputs])
decoder_concat = tf.keras.layers.Concatenate()([decoder_outputs, attention])

# Couche de sortie
decoder_dense = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat)

# Modèle complet
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [24]:
# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(input_seqs, target_seqs, test_size=0.2)

# Les entrées du décodeur sont les réponses sans le dernier token
decoder_input_data = y_train[:, :-1]
# Les sorties du décodeur sont les réponses décalées d'un token
decoder_output_data = y_train[:, 1:]

# Idem pour les données de test
decoder_input_test = y_test[:, :-1]
decoder_output_test = y_test[:, 1:]


In [25]:
from tensorflow.keras.metrics import Metric
import tensorflow as tf

class Perplexity(Metric):
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total_log_prob = self.add_weight(name='log_prob', initializer='zeros')
        self.total_words = self.add_weight(name='words', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # 1. Masque pour ignorer le padding (token 0)
        mask = tf.cast(y_true != 0, tf.float32)
        num_words = tf.reduce_sum(mask)

        # 2. Log-probabilités des tokens cibles (stable numériquement)
        y_pred = tf.clip_by_value(y_pred, 1e-10, 1.0)
        log_probs = tf.math.log(tf.gather(y_pred, y_true, axis=-1, batch_dims=2))

        # 3. Mise à jour des compteurs
        self.total_log_prob.assign_add(tf.reduce_sum(log_probs * mask))
        self.total_words.assign_add(num_words)

    def result(self):
        return tf.exp(-self.total_log_prob / self.total_words)

    def reset_state(self):
        self.total_log_prob.assign(0.0)
        self.total_words.assign(0.0)

In [26]:
perplexity_metric = Perplexity()
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=[perplexity_metric, accuracy_metric]  # Ajout ici
)

In [27]:
import os

if os.path.exists('chatbot_model.h5'):
    print("Model already trained. Loading the model...")
    model.load_weights('chatbot_model.h5')
else:
    print("Training the model...")
    history = model.fit(
        [X_train, decoder_input_data],
        np.expand_dims(decoder_output_data, -1),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=([X_test, decoder_input_test], np.expand_dims(decoder_output_test, -1)),
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=3),
            tf.keras.callbacks.ModelCheckpoint('chatbot_model.h5', save_best_only=True)
        ]
    )


Model already trained. Loading the model...


In [28]:
# Encodeur pour l'inférence
encoder_model = Model(encoder_inputs, [encoder_outputs, encoder_states])

# Décodeur pour l'inférence
decoder_state_input_h = Input(shape=(LSTM_UNITS*2,))
decoder_state_input_c = Input(shape=(LSTM_UNITS*2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

# Attention pour l'inférence
attention = Attention()([decoder_outputs, encoder_outputs])
decoder_concat = tf.keras.layers.Concatenate()([decoder_outputs, attention])

decoder_outputs = decoder_dense(decoder_concat)
decoder_model = Model(
    [decoder_inputs] + [encoder_outputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)


In [29]:
def generate_response(input_seq):
    # Encoder l'input
    enc_out, enc_states = encoder_model.predict(input_seq)

    # Démarrer avec le token <start>
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word_index['<start>']

    stop_condition = False
    response = []

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [enc_out] + enc_states)

        # Sélectionner le token avec la plus haute probabilité
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = None

        for word, index in word_index.items():
            if index == sampled_token_index:
                sampled_word = word
                response.append(word)

        # Condition d'arrêt: <end> ou longueur max atteinte
        if sampled_word == '<end>' or len(response) >= MAX_LENGTH:
            stop_condition = True

        # Mettre à jour la séquence cible
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Mettre à jour les états
        enc_states = [h, c]

    return ' '.join(response[:-1])  # Exclure le token <end>

In [37]:
# prompt: appeal generate_response with sentence "how are you?"

input_sentence = "hi honey!"
input_seq = tokenizer.texts_to_sequences([clean_text(input_sentence)])
input_seq = pad_sequences(input_seq, maxlen=MAX_LENGTH, padding='post')
response = generate_response(input_seq)
response

1/1 [==============================] - 0s 29ms/step


'hello .'

In [38]:
# prompt: can you generate a tkinter interface in which the user puts a sentence and get the response on his screen?

import tkinter as tk
from tkinter import scrolledtext

# ... (Your existing code for model loading and response generation) ...

def get_response():
    input_sentence = input_text.get("1.0", tk.END).strip()
    input_seq = tokenizer.texts_to_sequences([clean_text(input_sentence)])
    input_seq = pad_sequences(input_seq, maxlen=MAX_LENGTH, padding='post')
    response = generate_response(input_seq)
    output_text.delete("1.0", tk.END)
    output_text.insert(tk.END, response)

root = tk.Tk()
root.title("Chatbot Interface")

input_label = tk.Label(root, text="Enter your sentence:")
input_label.pack(pady=5)

input_text = tk.Text(root, height=3, width=50)
input_text.pack(pady=5)

get_response_button = tk.Button(root, text="Get Response", command=get_response)
get_response_button.pack(pady=10)

output_label = tk.Label(root, text="Chatbot Response:")
output_label.pack()

output_text = scrolledtext.ScrolledText(root, height=10, width=50, wrap=tk.WORD)
output_text.pack(pady=5)


root.mainloop()


1/1 [==============================] - 0s 24ms/step
